In [1]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Writing add.cpp


In [ ]:
!apt-get install -y libopencv-dev python3-opencv

In [2]:
%%shell
g++ add.cpp -o add

In [3]:
%%shell
./add

Max error: 0


In [21]:
%%writefile include/utils.h

#ifndef UTILS_H
#define UTILS_H

#include <math.h>
#include <opencv2/opencv.hpp>

namespace matrix {
// Function to compare two matrices
inline bool compare_matrices(const float* mat1, const float* mat2, int rows, int cols, float epsilon = 1e-4) {
    for (int i = 0; i < rows * cols; ++i) {
        if (std::abs(mat1[i] - mat2[i]) > epsilon) {
            return false;
        }
    }
    return true;
}
}

namespace utils{
// Function to compare two matrices
inline bool compare_vectors(const float* vec1, const float* vec2, int size, float epsilon = 1e-4) {
    for (int i = 0; i < size; ++i) {
        if (std::abs(vec1[i] - vec2[i]) > epsilon) {
            return false;
        }
    }
    return true;
}
}

namespace imageio {
inline cv::Mat readImage(std::string image_path) {
    cv::Mat image = cv::imread(image_path, cv::IMREAD_COLOR);

    if (image.empty()) {
        std::cerr << "Could not open or find the image at " << image_path << std::endl;
        exit(EXIT_FAILURE);
    }
    return image;
}

inline cv::Mat toFloatMat(const cv::Mat& mat) {
    cv::Mat float_mat;
    if (mat.type() != CV_32F) {
        mat.convertTo(float_mat, CV_32F);
    } else {
        float_mat = mat;
    }
    return float_mat;
}

inline void matToFloatArray(const cv::Mat& mat, float* out, size_t size) {
    cv::Mat floatMat = toFloatMat(mat);
    std::memcpy(out, floatMat.data, size * sizeof(float));
}

inline cv::Mat floatArrayToMat(const float* floatArray, int width, int height, int channels) {
    cv::Mat image(height, width, CV_32FC(channels));
    std::memcpy(image.data, floatArray, width * height * channels * sizeof(float));
    return image;
}

inline void writeImage(const float* float_array, int width, int height, int num_channels, std::string output_path) {
// convert float array to matrix and write to disk.
    cv::Mat restored = imageio::floatArrayToMat(float_array, width, height, num_channels);
    bool success = cv::imwrite(output_path, restored);
    if (!success) {
        std::cerr << "Could not write image to disk at " << output_path << std::endl;
        exit(EXIT_FAILURE);
    }
}

}

#endif // UTILS_H%

Writing include/utils.h


In [27]:
%%writefile include/matrix.h

#ifndef MATRIX_H
#define MATRIX_H

#include <memory>
#include <functional>
#include <iomanip>

struct Matrix {
    int height;
    int width;
    std::unique_ptr<float[]> data;
    Matrix(int height_, int width_, std::function<void(float*, int)> init_func)
    : height(height_), width(width_), data(std::make_unique<float[]>(height_ * width_)) {
        init_func(data.get(), height_ * width_);
    }

    // copy constructor, deep copies data value.
    // reason for this is that we support in-place operations on matrix
    // having this just makes it easier to validate the cpu vs gpu implementations.
    Matrix(const Matrix& other)
    : height(other.height), width(other.width),
    data(std::make_unique<float[]>(other.height * other.width)) {
        std::copy(other.data.get(), other.data.get() + other.height * other.width, data.get());
    }

    // TODO: add move semantics

    void print() const {
        std::cout << std::fixed << std::setprecision(4);
        for(int i =0; i < height; ++i) {
            for(int j = 0; j < width; ++j) {
                std::cout << data[i * width + j] << " ";
            }
            std::cout << std::endl;
        }
    }
};

#endif //MATRIX_H

Overwriting include/matrix.h


In [28]:
%%writefile include/init_utils.h

#ifndef INIT_UTILS_H
#define INIT_UTILS_H

#include <random>

inline void random_init(float *array, int size) {
    std::random_device rd;
    std::mt19937 gen(rd());

    std::uniform_real_distribution<> dist(0.0, 1.0);

    for(int i=0; i < size; ++i) {
        array[i] = dist(gen);
    }
}

#endif // INIT_UTILS_H

Overwriting include/init_utils.h


In [29]:
%%writefile include/cuda_utils.h

#ifndef CUDA_UTILS_H
#define CUDA_UTILS_H

#include <cuda_runtime.h>
#include <iostream>

#define CUDA_ERROR_CHECK(call)  { \
    cudaError_t error = call; \
    if (error != cudaSuccess) { \
        fprintf(stderr, "CUDA error in file '%s' in line %i.%s \n", \
                __FILE__, __LINE__, cudaGetErrorString(error)); \
        exit(EXIT_FAILURE); \
    }}

#define TIMED_CUDA_FUNCTION() CudaEventTimer(__FUNCTION__)

class CudaEventTimer {

public:
    CudaEventTimer(const char* function_name):
    function_name_(function_name) {
        cudaEventCreate(&start_);
        cudaEventCreate(&stop_);
        cudaEventRecord(start_);
    }

    ~CudaEventTimer() {
        cudaEventRecord(stop_);
        cudaEventSynchronize(stop_);
        float milliseconds = 0.0f;
        cudaEventElapsedTime(&milliseconds, start_, stop_);
        std::cout << "CUDA function " << function_name_ << " finished in " << milliseconds << " ms" << std::endl;
        cudaEventDestroy(start_);
        cudaEventDestroy(stop_);

    }
private:
    const char* function_name_;
    cudaEvent_t start_, stop_;
};

#endif // CUDA_UTILS_H

Overwriting include/cuda_utils.h


In [30]:
%%writefile include/iming_utils.h

#ifndef TIMING_UTILS_H
#define TIMING_UTILS_H

#include <chrono>
#include <string>
#include <iostream>

#define TIMED_CPU_FUNCTION() timers::FunctionTimer timer(__FUNCTION__)

namespace timers {
class FunctionTimer {
public:
    FunctionTimer(std::string function_name) :
    function_name_(function_name), start_(std::chrono::high_resolution_clock::now()) {}

    ~FunctionTimer() {
        auto end = std::chrono::high_resolution_clock::now();
        auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - start_).count();
        std::cout << "CPU function " << function_name_ << " finished in " << duration << " ms" <<std::endl;
    }

private:
std::string function_name_;
std::chrono::time_point<std::chrono::high_resolution_clock> start_;

};
}

#endif //TIMING_UTILS_H%

Writing include/iming_utils.h


In [31]:
%%writefile include/cpu_kernels.h

#ifndef CPU_KERNELS_H
#define CPU_KERNELS_H

#include <vector>

void matmul_cpu(const float* a, const float* b, float* c, int M, int K, int N);
void conv1d_cpu(const std::vector<float>& matrix, const std::vector<float>& conv_mask,
                std::vector<float>& output, int mask_width, int width);

void conv2d_cpu(const std::vector<float>& matrix, const std::vector<float>& conv_mask,
                std::vector<float>& output, int r, int width, int height);

void softmax_cpu(float* mat, int M, int N);


#endif // CPU_KERNELS_H%

Overwriting include/cpu_kernels.h


In [7]:
%%writefile cpu_kernels.cpp

#include "cpu_kernels.h"
#include "timing_utils.h"

void matmul_cpu(const float* a, const float* b, float* c, int M, int K, int N) {
    TIMED_CPU_FUNCTION();
    for(int row=0; row < M; ++row) {
        for(int col = 0; col < N; ++col) {
            float value = 0.0f;
            for (int k = 0; k < K; ++k) {
                value += a[row * K + k] * b[k * N + col];
            }
            c[row * N + col] = value;
        }
    }
}

Writing cpu_kernels.cpp


In [32]:
%%writefile include/gemm.h

#ifndef GEMM_H
#define GEMM_H

#define TILE_WIDTH 16

#include <cuda_runtime.h>

__global__ void gemm_cuda_tiled(const float* __restrict__  A, const float* __restrict__ B, float* C, int M, int K, int N);


#endif // GEMM_H

Overwriting include/gemm.h


In [45]:
%%writefile gemm.cu

#include "timing_utils.h"
#include "init_utils.h"
#include "matrix.h"
#include "cuda_utils.h"
#include "gemm.h"
#include "cpu_kernels.h"
#include "utils.h"

#include <iostream>
#include <cuda_runtime.h>
#include <iomanip>

__global__ void gemm_cuda_tiled(const float* __restrict__  a, const float* __restrict__ b,
                                float* c, int M, int K, int N) {
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;

    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    __shared__ float a_shared[TILE_WIDTH][TILE_WIDTH];
    __shared__ float b_shared[TILE_WIDTH][TILE_WIDTH];

    float p_value = 0.0f;
    for(int ph = 0; ph < (K + TILE_WIDTH - 1)/ TILE_WIDTH; ++ph) {
        if (row < M && ph * TILE_WIDTH + tx < K) {
            a_shared[ty][tx] = a[row * K + ph * TILE_WIDTH + tx];
        } else {
            a_shared[ty][tx] = 0.0f;
        }
        if(ph * TILE_WIDTH + ty < K && col < N) {
            b_shared[ty][tx] = b[(ph * TILE_WIDTH + ty) * N + col];
        } else {
            b_shared[ty][tx] = 0.0f;
        }

        __syncthreads();

        for(int i = 0; i < TILE_WIDTH; ++i) {
            p_value += a_shared[ty][i] * b_shared[i][tx];
        }
        __syncthreads();
    }
    if(row < M && col < N) {
        c[row * N + col] = p_value;
    }
}

void gemm_kernel_launch(float* mat1_d, float* mat2_d, float* out_d, int M, int K, int N) {
    TIMED_CUDA_FUNCTION();
    int block_size_x = TILE_WIDTH;
    int block_size_y = TILE_WIDTH;

    dim3 threads_per_block(block_size_x,
                           block_size_y);

    dim3 blocks_per_grid((N + block_size_x - 1) / block_size_x,
                         (M + block_size_y - 1) / block_size_y);

    gemm_cuda_tiled<<<blocks_per_grid, threads_per_block>>>(mat1_d, mat2_d, out_d, M, K, N);
    cudaDeviceSynchronize();
}

int main() {

    int M = 128;
    int K = 64;
    int N = 128;
    Matrix mat1_h(M, K, random_init);
    Matrix mat2_h(K, N, random_init);
    float* out_h = new float[M * N];
    float* out_cpu = new float[M * N];
    matmul_cpu(mat1_h.data.get(), mat2_h.data.get(), out_cpu, M, K, N);
    float *mat1_d, *mat2_d, *out_d;

    CUDA_ERROR_CHECK(cudaMalloc((void**) &mat1_d, M * K * sizeof(float)));
    CUDA_ERROR_CHECK(cudaMalloc((void**) &mat2_d, K * N * sizeof(float)));
    CUDA_ERROR_CHECK(cudaMalloc((void**) &out_d, M * N * sizeof(float)));

    CUDA_ERROR_CHECK(cudaMemcpy(mat1_d, mat1_h.data.get(), M * K * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_ERROR_CHECK(cudaMemcpy(mat2_d, mat2_h.data.get(), K * N * sizeof(float), cudaMemcpyHostToDevice));

    gemm_kernel_launch(mat1_d, mat2_d, out_d, M, K, N);

    CUDA_ERROR_CHECK(cudaMemcpy(out_h, out_d, M * N * sizeof(float), cudaMemcpyDeviceToHost));

    if (matrix::compare_matrices(out_h, out_cpu, M, N)) {
        std::cout << "CUDA kernel's result matches the CPU result." << std::endl;
    } else {
        std::cerr << "CUDA kernel's result does NOT match the CPU result." << std::endl;
    }

    delete [] out_h;
    delete [] out_cpu;
    cudaFree(mat1_d);
    cudaFree(mat2_d);
    cudaFree(out_d);
    return 0;
}


Overwriting gemm.cu


In [46]:
%%shell
nvcc -I/usr/include/opencv4 -I./include -L/usr/lib gemm.cu cpu_kernels.cpp -o gemm `pkg-config --cflags --libs opencv4` -diag-suppress=611

In [47]:
%%shell
./gemm

CPU function matmul_cpu finished in 3 ms
CUDA function gemm_kernel_launch finished in 0.002048 ms
CUDA kernel's result matches the CPU result.
